In [17]:
import tensorflow as tf

import magenta

from magenta.music import performance_lib

from magenta.protobuf import music_pb2

from magenta.pipelines import pipeline
from magenta.pipelines import pipelines_common
from magenta.pipelines import dag_pipeline
from magenta.pipelines import note_sequence_pipelines
from magenta.pipelines import statistics

from magenta.pipelines.pipeline import _guarantee_dict

import os

In [18]:
# Defaults to 0, all logs are shown. 
# 1 to filter out INFO logs
# 2 to additionall filter out WARNING
# 3 to additionally filter out ERROR
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

# Create Dataset

This will run data through a few pipelines.

**POINTERS**
* `perf_extractor` converts a `NoteSequence` of MIDI-like sequence events (`NOTE ON`, `NOTE OFF`) to a `music.Performance` additionally encoding TIME SHIFT events.

**PIPELINE PARAMETERS**

In [26]:
pipelines_config = dict()

pipelines_config['data_source_dir'] = "./data/note_seq_proto/"
pipelines_config['data_target_dir'] = "./data/performance_seq_text/"

pipelines_config['steps_per_second'] = 100

pipelines_config['min_events'] = 1
pipelines_config['max_events'] = 10000

pipelines_config['eval_ratio'] = 0.1
pipelines_config['test_ratio'] = 0.1



### Method Definitions

In [20]:
class PerformanceExtractor(pipeline.Pipeline):
    """Extracts polyphonic tracks from a quantized NoteSequence."""

    def __init__(self, min_events, max_events, num_velocity_bins, name=None):
        super(PerformanceExtractor, self).__init__(
            input_type=music_pb2.NoteSequence,
            output_type=magenta.music.Performance,
            name=name)
        self._min_events = min_events
        self._max_events = max_events
        self._num_velocity_bins = num_velocity_bins

    def transform(self, quantized_sequence):
        performances, stats = magenta.music.extract_performances(
            quantized_sequence,
            num_velocity_bins=self._num_velocity_bins)
        self._set_stats(stats)
        return performances
    
class PerformanceParser(pipeline.Pipeline):
    """Converts a Performance into a text sequence.
    
    Individual events become 'words' of A-Z 0-9 separated by space. 
    """
    
    def __init__(self, name=None):
        super(PerformanceParser, self).__init__(
            input_type=magenta.music.Performance,
            output_type=str,
            name=name)
        
    def transform(self, performance):
        strs = []
        for event in performance:
            if event.event_type == performance_lib.PerformanceEvent.NOTE_ON:
                strs.append('ON%s' % event.event_value)
            elif event.event_type == performance_lib.PerformanceEvent.NOTE_OFF:
                strs.append('OFF%s' % event.event_value)
            elif event.event_type == performance_lib.PerformanceEvent.TIME_SHIFT:
                strs.append('SHIFT%s' % event.event_value)
            else:
                raise ValueError('Unknown event type: %s' % event.event_type)
        return [' '.join(strs)]


In [21]:
def run_pipeline_text(pipeline,
                      input_iterator,
                      output_dir):
    """Runs a pipeline graph saving output to disk as text.
     
    Run the the pipeline on each input from the iterator one at a time.
    A file will be written to `output_dir` for each dataset name specified
    by the pipeline. pipeline.transform is called on each input and the
    results are aggregated into their correct datasets.

    The output type given by `pipeline.output_type` must be str.

    Args:
        pipeline: A Pipeline instance. `pipeline.output_type` must be a str.
        input_iterator: Iterates over the input data. Items returned by it are fed
            directly into the pipeline's `transform` method.
        output_dir: Path to directory where datasets will be written. Each dataset
            is a file whose name contains the pipeline's dataset name. If the
            directory does not exist, it will be created.
            
    Raises:
        ValueError: If any of `pipeline`'s output type is not str.
     
    """
    
    if isinstance(pipeline.output_type, dict):
        for name, type_ in pipeline.output_type.items():
            if type_ != str:
                raise ValueError(
                    'Pipeline "%s" must output %s type. '
                    'Output type was %s' % (name, str, type_))
    else:
         if type_ != str:
            raise ValueError(
                    'Pipeline "%s" must output %s type. '
                            'Output type was %s' % (name, str, pipeline.output_type))
    
    
    aggregated_outputs = dict(
        [(name, []) for name in pipeline.output_type_as_dict])
    total_inputs = 0
    total_outputs = 0
    stats = []
    
    output_names = pipeline.output_type_as_dict.keys()
    output_paths = [os.path.join(output_dir, name + '.txt')
                    for name in output_names]

    for path in output_paths:
        if os.path.exists(path):
            raise FileExistsError('File {} already exists. Please remove and try again.'
                                        .format(path))           

    writers = dict([(name, open(path, 'a'))
                  for name, path in zip(output_names, output_paths)])
    
    for input_object in input_iterator:
        total_inputs += 1
        
        for name, outputs in _guarantee_dict(
            pipeline.transform(input_object),
            list(output_names)[0]).items():
            
            for output in outputs:
                writers[name].write(output + '\n')
                
            total_outputs += len(outputs)
        stats = statistics.merge_statistics(stats + pipeline.get_stats())
        if total_inputs % 5000 == 0:
            tf.logging.info('Processed %d inputs so far. Produced %d outputs.', 
                            total_inputs, total_outputs)
            statistics.log_statistics_list(stats, tf.logging.info)
    tf.logging.info('\n\nCompleted.\n')
    tf.logging.info('Processed %d inputs total. Produced %d outputs.',
                    total_inputs, total_outputs)
    statistics.log_statistics_list(stats, tf.logging.info)
    return aggregated_outputs


In [22]:
def build_pipeline_graph(collection_name,
                         config):
    """Returns the Pipeline instance which creates the RNN dataset.

    Args:
        collection_name:
        config: dict() with configuration settings

    Returns:
        A pipeline.Pipeline instance.
    """
    
    # Stretch by -5%, -2.5%, 0%, 2.5%, and 5%.
    stretch_factors = [0.95, 0.975, 1.0, 1.025, 1.05]

    # Transpose no more than a major third.
    transposition_range = range(-3, 4)

    partitioner = pipelines_common.RandomPartition(
        music_pb2.NoteSequence,
        ['eval_arrangement' + '_' + collection_name, 
         'test_arrangement' + '_' + collection_name, 
         'train_arrangement' + '_' + collection_name],
        [pipelines_config['eval_ratio'], pipelines_config['test_ratio']])
    dag = {partitioner: dag_pipeline.DagInput(music_pb2.NoteSequence)}

    for mode in ['eval', 'test', 'train']:
        key = mode + '_arrangement' + '_' + collection_name
        
        quantizer = note_sequence_pipelines.Quantizer(
            steps_per_second=pipelines_config['steps_per_second'], 
            name='Quantizer_' + key)
        
        perf_extractor = PerformanceExtractor(
            min_events=pipelines_config['min_events'],
            max_events=pipelines_config['max_events'],
            num_velocity_bins=0,
            name='PerformanceExtractor_' + key)
            # input_type = music_pb2.NoteSequence
            # output_type = magenta.music.Performance
            
        perf_parser = PerformanceParser(
            name='PerformanceParser_' + key)
            # input_type = magenta.music.Performance
            # output_type = str
        
        dag[quantizer] = partitioner[key]
        dag[perf_extractor] = quantizer
        dag[perf_parser] = perf_extractor
        dag[dag_pipeline.DagOutput(key)] = perf_parser
        
    return dag_pipeline.DAGPipeline(dag)

In [23]:
def create_dataset():
    for collection_name in ['inputs', 'targets']:
        
        src_file = os.path.join(pipelines_config['data_source_dir'], 
                                   collection_name + '.tfrecord')
        output_dir = pipelines_config['data_target_dir']
        
        # Construct the pipeline graph
        pipeline_graph = build_pipeline_graph(
            collection_name = collection_name,
            config = pipelines_config
        )

        # Runs pipeline graph on a data source and writes output to dir
        run_pipeline_text(
            pipeline_graph,
            pipeline.tf_record_iterator(src_file,
                                        pipeline_graph.input_type),
            output_dir
        )

In [38]:
def build_vocab():
    
    file_path = pipelines_config['data_target_dir'] + 'vocab.txt'
    
    vocab = {
        'ON': (0, 127 + 1),
        'OFF': (0, 127 + 1),
        'SHIFT': (0, pipelines_config['steps_per_second'] + 1),
    }  
    
    if os.path.exists(file_path):
        print("INFO: File {} exists. Removing. Rebuilding vocabulary.".format(file_path))
        os.remove(file_path)
    with open(file_path, 'a') as file:
        for action in vocab.keys():
            for val in range(vocab[action][0], vocab[action][1]):
                file.write(action + str(val) + '\n')
                
    print("INFO: Vocabulary built.")

# Run below to build your dataset

This will create 6 files, two sets of train, evaluate and test. The first set is the inputs, and the second set is the targets.

In [25]:
create_dataset()

INFO:tensorflow:

Completed.

INFO:tensorflow:Processed 88 inputs total. Produced 88 outputs.
INFO:tensorflow:DAGPipeline_PerformanceExtractor_eval_arrangement_inputs_performance_lengths_in_seconds:
  [20,30): 3
  [30,40): 4
  [40,60): 1
  [60,120): 3
INFO:tensorflow:DAGPipeline_PerformanceExtractor_eval_arrangement_inputs_performances_discarded_more_than_1_program: 0
INFO:tensorflow:DAGPipeline_PerformanceExtractor_eval_arrangement_inputs_performances_discarded_too_short: 0
INFO:tensorflow:DAGPipeline_PerformanceExtractor_eval_arrangement_inputs_performances_truncated: 0
INFO:tensorflow:DAGPipeline_PerformanceExtractor_eval_arrangement_inputs_performances_truncated_timewise: 0
INFO:tensorflow:DAGPipeline_PerformanceExtractor_test_arrangement_inputs_performance_lengths_in_seconds:
  [5,10): 1
  [10,20): 1
  [30,40): 5
  [40,60): 4
  [60,120): 2
INFO:tensorflow:DAGPipeline_PerformanceExtractor_test_arrangement_inputs_performances_discarded_more_than_1_program: 0
INFO:tensorflow:DAGPipel

In [39]:
build_vocab()

INFO: File ./data/performance_seq_text/vocab.txt exists. Removing. Rebuilding vocabulary.
INFO: Vocabulary built.


### Helpful IO

##### Read `.tfrecord` of NoteSequences.

In [17]:
tfrecord_path = './data/note_seqs/inputs.tfrecord'

In [120]:
records = list()
for record in pipeline.tf_record_iterator(tfrecord_path, music_pb2.NoteSequence):
    records.append(record)

# Hint: To look at first record, run `records[0]`
# Hint 2: This is useful to drill inside: records[0].feature_lists.feature_list['inputs'].feature

##### Instantiate pipelines

In [121]:
from magenta.models.performance_rnn import performance_model
from magenta.pipelines import note_sequence_pipelines

config = constants.default_configs['performance']

quantizer_instance = note_sequence_pipelines.Quantizer(steps_per_second = config.steps_per_second,
                                                       name='Quantizer_jupyter')
perf_extractor_instance = PerformanceExtractor(num_velocity_bins = config.num_velocity_bins)
encoder_pipeline_instance = EncoderPipeline(config,
                                            name='EncoderPipeline_jupyter')

##### Step-through pipelines

In [198]:
for record in pipeline.tf_record_iterator(tfrecord_path, music_pb2.NoteSequence):
    note_sequence1 = record
# note_sequence1.SerializeToString()
# note_sequence1

In [180]:
note_sequence2 = quantizer_instance.transform(note_sequence1)[0]

In [563]:
note_sequence3 = perf_extractor_instance.transform(note_sequence2)[0]

# note_sequence3.to_sequence()        # converts Performance to NoteSequence proto
# note_sequence3.__getitem__(4)       # return event at position
# note_sequence3.steps_per_second     # if Performance(BasePerformance)
# note_sequence3.steps_per_quarter    # if MetricPerformance(BasePerformance)
# note_sequence3._events              # list of all events
# len(note_sequence3._events)         # len of events
# note_sequence3.num_steps            # len of sequence in quantized steps
# note_sequence3.steps                # list of the time step at each event in the sequence

# returns an iterator
# for i, event in enumerate(note_sequence3.__iter__()): 
#     print(event)
#     if i > 25:
#         break

In [261]:
note_sequence4 = encoder_pipeline_instance.transform(note_sequence3)[0]
# note_sequence4

#### Determining which position is 1 in the one-hot vector

In [206]:
_event_ranges = [
    (1, 1, 127),
    (2, 1, 127),
    (3, 1, 100)
]

def encode_event(event):
    offset = 0
    for event_type, min_value, max_value in _event_ranges:
        if event[0] == event_type:
            return offset + event[1] - min_value
        offset += max_value - min_value + 1

encode_event((3, 100))

353

#### Investigate time-shifting